In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import dframe_tools as dtools
import plot_tools as ptools
import gen_resubmit as gensub

# Comment out for description of below modules
# dtools.help()
# ptools.help()
# gensub.help()

'''
Assign path of OUTPUT files and generate database
Options:
skip = int or float [int: number of line to skip, float: proportion to skip]
noenergy = 0 if you have energy estimator, 1 if you don't. 0: pick 9, 10th column 1: pick 0, 1st column
replace = 1 if you want to replace nan to zero. Be cautious to use it
'''

path = '/home/sang/UVM/He4vacuum/OUTPUT'
dbase = dtools.df_pimc(path, skip = 0, noenergy = 1, replace = 1)

dfdict = ptools.datadic(dbase)

In [ ]:
# sorting for key
def keysorting(k):
    t = k.split(',')
    n = float(t[0])*100000 + float(t[1])
    return n

keylst = []
# Listing keys to ban
banlst = []
# banlst = ['0.03,-103.0','0.04,-103.0','0.05,-103.0', '0.06,-103.0', '0.07,-103.0']

for keys in dfdict.keys():
    if keys not in banlst:
        keylst.append(keys)
keylst = sorted(keylst, key = lambda k: keysorting(k))

# Check sorted list of keys
print(keylst)
print('number of category:', len(keylst))

# nparallel check
'''return number of copies with same configurations in the subset dataframes
    if the total # of data is n times total # of (T, totN) combination.
    return None if it's something else'''

for keys in keylst:
    print(ptools.nparallel( dfdict[keys] ) , end='  ')

In [ ]:
# Initialize outputs
case_out = {}
case_nmean = {}
case_nerr = {}
case_alist = {}
case_aerrlist = {}

# calculate extrapolated data without plotting them
for k in keylst:
    case_out[k] = ptools.esti_array_multi(dfdict[k])
    case_nmean[k], case_nerr[k] = ptools.non_plot_frac(case_out[k])
    case_alist[k], case_aerrlist[k]  = ptools.non_plot_superfrac(case_out[k])

In [ ]:
# If you want to plot data extrapolating, use below
# case_key = '0.05,-101.0'
# case_out[case_key] = ptools.esti_array_multi(dfdict[case_key])
# case_nmean[case_key], case_nerr[case_key] = ptools.plot_frac(case_out[case_key])
# case_alist[case_key], case_aerrlist[case_key]  = ptools.plot_superfrac(case_out[case_key])

In [ ]:
# use following functions to SF vs temperature plot of set of keys
def features_T(row):
    
    area = np.sqrt(3)/4*6*1.42
    mass = 10*6.7

    T_lists = [case_out[k]['Tset'] for k in row]
    f_lists = [np.asarray(case_nmean[k])[:,0] for k in row]
    ferr_lists = [np.asarray(case_nerr[k])[:,0] for k in row]
    as_lists = [np.asarray(case_alist[k])[:,0] for k in row]
    as_err_lists = [np.asarray(case_aerrlist[k])[:,0] for k in row]
    rhos_lists = [np.asarray(case_nmean[k])[:,0]*np.asarray(case_alist[k])[:,0]*mass/area for k in row]
    rhos_err_lists = [np.asarray(case_nmean[k])[:,0]*np.asarray(case_aerrlist[k])[:,0]*mass/area for k in row]
    
    return T_lists, f_lists, ferr_lists, as_lists, as_err_lists, rhos_lists, rhos_err_lists

def final_plot(rows):
    linex = np.linspace(0,1.5,10)
    colmax = max([len(row) for row in rows])
    fig, axs = plt.subplots(colmax,len(rows),sharex=True, sharey=True, figsize=[2*len(rows),(1.8)*colmax])
#     plt.setp(axs, xlim=(0,1.5), ylim=(0.2,0.8))

    for j,row in enumerate(rows):
        T_lists, f_lists, ferr_lists, as_lists, as_err_lists, rhos_lists, rhos_err_lists = features_T(row)
        
        for i in range(len(row)):
            ax1 = axs[i,j]
            ax2 = ax1.twinx()
#             ax1.errorbar(T_lists[i],rhos_lists[i], yerr = rhos_err_lists[i], 
#                          color = 'brown', label = row[i]+'vs1/sqrt(N)', elinewidth = 5, ecolor = 'grey')
            ax1.errorbar(T_lists[i],as_lists[i], yerr = as_err_lists[i], 
                         color = 'brown', label = row[i]+'vs1/sqrt(N)', elinewidth = 5, ecolor = 'grey')
            ax1.set_ylim(0,1)
            ax1.legend()
#             axs[i,j].plot(linex,7.6/2.2*linex, label = 'KT static theory')
            ax2.errorbar(T_lists[i],f_lists[i], yerr = ferr_lists[i], fmt = '-',
                         color = 'green', elinewidth = 5, ecolor = 'orange')
            ax2.set_ylim(0.2,0.8)
            ax2.tick_params(axis='y', colors='green')
            if row != rows[-1]:
                ax2.set_yticks(np.arange(0.2,0.8,0.1))
                ax2.set_yticklabels(['' for r in np.arange(0.2,0.8,0.1)])
            
#     fig.text(0.5, 0.0, r'T (K)', ha='center', fontsize=12)
#     fig.text(0.0, 0.5, r'SF density$\times 10^{9} \,$ ($\text{g cm}^{-2}$)', 
#              va='center', rotation='vertical', fontsize=12)
#     fig.text(1.0, 0.5, 'filling', va='center', rotation=-90, fontsize=12)
    fig.tight_layout()

# define keys_row1, keys_row2,keys_row3,keys_row4,
#final_plot((keys_row1,keys_row2,keys_row3,keys_row4))

In [ ]:
# Find critical temperature
def features_T_each(k):
    
    area = np.sqrt(3)/4*6*1.42
    mass = 10*6.7

    T_lists = case_out[k]['Tset']
    f_lists = np.asarray(case_nmean[k])[:,0]
    ferr_lists = np.asarray(case_nerr[k])[:,0]
    as_lists = np.asarray(case_alist[k])[:,0]
    as_err_lists = np.asarray(case_aerrlist[k])[:,0]
    rhos_lists = np.asarray(case_nmean[k])[:,0]*np.asarray(case_alist[k])[:,0]*mass/area
    rhos_err_lists = np.asarray(case_nmean[k])[:,0]*np.asarray(case_aerrlist[k])[:,0]*mass/area
    
    return T_lists, f_lists, ferr_lists, as_lists, as_err_lists, rhos_lists, rhos_err_lists

# These are other functions to find Tc, but they are deprecated
def find_Tc(new_x, data):
    Ts = []
    Rs = data[0][::-1]
    triger = -1
    stopper = 0
    for i, v in enumerate(tuple(Rs)):
        if v > 0.1:
            triger = i
            break
    while triger != -1 and stopper == 0:
        checker = 0
        if triger+1 == len(Rs):
            stopper = 1
            break
        for i, v in enumerate(tuple(Rs[triger+1:])):
            if v < 0.1:
                triger += i+1
                break
            else:
                checker += 1
        if checker >= len(Rs)-1-triger:
            stopper = 1
            break
        
                
                
    for i, v in enumerate(tuple(Rs[triger:])):
        if v > 0.1:
            Ts.append(new_x[-1*(i+triger)])
            break
    for i, v in enumerate(tuple(Rs[triger:])):
        if v > 0.2:
            Ts.append(new_x[-1*(i+triger)])
            break
            
    if len(Ts) < 2 :
        return 0  
    else:
        return Ts[0]

def find_Tc_old(new_x, data):
    Ts = []
    Rs = data[0][::-1]
    triger = -1
    stopper = 0
    for i, v in enumerate(tuple(Rs)):
        if v > 0.2:
            Ts.append(new_x[-1*i])
            triger = i
            break
    for i, v in enumerate(tuple(Rs)):
        if v > 1.0:
            Ts.append(new_x[-1*i])
            break
    for i, v in enumerate(tuple(Rs)):
        if v > 1.5:
            Ts.append(new_x[-1*i])
            break
            
    if len(Ts) < 3 :
        return 0
    else:
        return sum(Ts[:-1]) / len(Ts[:-1])
    
def find_root(new_x, Rset):
    Ts = []
    Rs = Rset[::-1]
    triger = -1
    stopper = 0
    for i, v in enumerate(tuple(Rs)):
        if v > 0.01:
            triger = i
            break
    while triger != -1 and stopper == 0:
        checker = 0
        if triger+1 == len(Rs):
            stopper = 1
            break
        for i, v in enumerate(tuple(Rs[triger+1:])):
            if v < 0.01:
                triger += i+1
                break
            else:
                checker += 1
        if checker >= len(Rs)-1-triger:
            stopper = 1
            break
        
                
                
    for i, v in enumerate(tuple(Rs[triger:])):
        if v > 0.01:
            Ts.append(new_x[-1*(i+triger)])
            break
    for i, v in enumerate(tuple(Rs[triger:])):
        if v > 0.1:
            Ts.append(new_x[-1*(i+triger)])
            break
            
    if len(Ts) < 2 :
        return 0  
    else:
        return Ts[0]

In [ ]:
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
import statistics

featuredata = {}

for k in keylst:
    T_lists, f_lists, ferr_lists, as_lists, as_err_lists, rhos_lists, rhos_err_lists = features_T_each(k)
    new_x = np.linspace(T_lists[0],T_lists[-1],50)
#     f1 = interp1d(T_lists, f_lists)
#     f2 = interp1d(T_lists, rhos_lists)
#     f2 = interp1d(T_lists, as_lists)
    weight = [1/x for x in T_lists]
    f2 = interp1d(T_lists, as_lists)
#     root1 = find_root(new_x, f2(new_x))
    f3 = UnivariateSpline(T_lists, as_lists, w=weight)
    f3.set_smoothing_factor(1.5)
    root2 = f3.roots()
    dev = np.sum(np.sqrt((f2(T_lists) - f3(T_lists))**2/4))
    
    dev *= (sum(T_lists[-2:])-sum(T_lists[:2]))/(sum(as_lists[-2:])-sum(as_lists[:2]))
    dev = np.sqrt(dev**2)*2
    
    featuredata[k] = [f3(new_x), T_lists, f_lists, ferr_lists, 
                      f_lists*as_lists, f_lists*as_err_lists, 
                      rhos_lists, rhos_err_lists, f2(new_x), as_lists, dev, root2]

In [ ]:
# Finds critical temperature and how much smoothing and actual are different
def find_Tc_new(data):
    Ts = []
    Rs = data[0]
    stdev = data[-2]
    root2 = data[-1]
    # when smooth plot is always above 0 and no root
    if len(root2) == 0:
        if Rs[-1] > 0:
            return 1.0, Rs[-1]
        else:
            return 0, 0
    # roots more than 1, but too low value for lowest temperature
    elif len(root2) > 1:
        if Rs[0] < 0.09:
            return 0, 0
        else:
#             return root2[0], np.sqrt((root2[0]-stdev)**2)
            return root2[0], stdev
    # root exist, but too low value for lowest temperature
    else:
        if Rs[0] < 0.09:
            return 0, 0
        else:
#             return root2[0], np.sqrt((root2[0]-stdev)**2)
            return root2[0], stdev

In [ ]:
# figure 1: mu-strain w/ Tc phase diagram
mu_s_Tc = []
mu_s_Tc_dev = []
new_x = np.linspace(0.6,1.0,50)

for k in keylst:
#     Tc = find_Tc(new_x, featuredata[k])
    Tc, Tc_dev = find_Tc_new(featuredata[k])
    mu_s_Tc.append(Tc)
    mu_s_Tc_dev.append(Tc_dev)
    
fig1 = plt.figure('fig1' ,figsize = [6,4])

ph_x = [float(k.split(',')[0])*100 for k in keylst] # strain
ph_y = [float(k.split(',')[1]) for k in keylst] # mu

plt.scatter(ph_x, ph_y, c = np.array(mu_s_Tc))
plt.fill_between([0,30], -120 + 1.30*np.array([0,30]), [-120,-120], color = 'lightgray')
plt.xlim(0,30)
plt.ylim(-120,-60)
plt.xlabel('strain (%)', fontsize=16)
plt.ylabel('chemical potential (K)', fontsize=16)
# plt.text(20, -110, 'Vacuum', fontsize = 24)
plt.hlines(-101.0, 1, 13, ls = '--', lw = 1.0, label = 'next figure')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
# plt.legend()
plt.show()

# Save?
# fig1.savefig('/home/sang/UVM/He4HighT/Allfigures/Tcs.png')

In [ ]:
# Export dot data?
# np.savetxt("/home/sang/UVM/He4vacuum/sfdots_x.csv", ph_x, delimiter =", ", fmt='%.12f')
# np.savetxt("/home/sang/UVM/He4vacuum/sfdots_y.csv", ph_y, delimiter =", ", fmt='%.12f')
# np.savetxt("/home/sang/UVM/He4vacuum/sfdots_Tc.csv", mu_s_Tc, delimiter =", ", fmt='%.12f')

In [ ]:
# figure 2: T-rhos w/ mu=-101 slice in diagram
# hkeys = []
# for k in keylst:
#     if k[-6:] == '-101.0':
#         hkeys.append(k)
# fig2 = plt.figure('fig2' ,figsize = [6,4])
# markers = ['ko-', 'ko-', 'ks-', 'k*-', 'kD-', 'kp-', 'kP-']
# Tcs = []
# strain = []
# for k in hkeys:
#     Tc = find_Tc(new_x, featuredata[k])
#     Tcs.append(Tc)
#     st = k[:4]
#     st = st.replace(',','0')
#     strain.append(st)

# for i, k in enumerate(hkeys):
#     plt.xlim(0.6, 1.0)
#     plt.ylim(0, 4)
#     xs = featuredata[k][1]
#     xs = np.array(xs)+0.005*(i-3.5)
#     ysl = featuredata[k][0]
#     ys = featuredata[k][-2]
#     yerrs = featuredata[k][-1]
# #     plt.bar(featuredata[k][1], featuredata[k][2], alpha = 0.5)
# #     plt.plot(new_x, ysl,'-')
#     plt.errorbar(xs, ys, yerr=yerrs, 
#                  fmt='.', ecolor=plt.get_cmap('tab10')(i), elinewidth=1, capsize=4, alpha = 0.7
#                 )
# #     plt.scatter(xs, ys, marker = 'd', c = colors[i*-1], s = np.array(yerrs)*1000, alpha = 0.5)
#     plt.plot(xs, ys, markers[i], 
#              color = plt.get_cmap('tab10')(i), mfc='none', 
#              linewidth=1.2, markersize=8, label=f'strain {int(float(strain[i])*100)} \%')
# #     plt.vlines(Tcs[i], 0, 4, color = colors[i*-1])
# plt.legend()
# plt.xlabel('Temperature (K)', fontsize=14)
# plt.ylabel(r'SF density$\times 10^{9} \,$ ($\text{g cm}^{-2}$)', fontsize=12)
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# plt.tight_layout()
# plt.show()
# fig2.savefig('newfig2.png')

In [ ]:
# figure 3: Tc-strain mu = -101 slice in diagram
cut_x = []
cut_y = []
cut_y_dev = []

for x, y, Tc, Tc_dev in zip(ph_x, ph_y, np.array(mu_s_Tc), np.array(mu_s_Tc_dev)):
    if y == -101.0 and int(x) != 14:
        cut_x.append(x)
        cut_y.append(Tc)
        cut_y_dev.append(Tc_dev)
        
f_Tc = UnivariateSpline(cut_x, cut_y)
f_Tc.set_smoothing_factor(0.03)
# new_x = np.linspace(0.03,0.1,20)
# new_x2 = np.arange(3,14.1,0.5)
# new_x3 = np.arange(3,13.4,0.5)
new_x3 = np.linspace(3,13.4,22)
fig3 = plt.figure('fig3', figsize = [4,3])
plt.fill_between(new_x3, f_Tc(new_x3), color='coral')
plt.fill_between([13.4,15.5], [2,2], color='grey')
pline = plt.plot(new_x3, f_Tc(new_x3), 'k-', linewidth=1.0, )
plt.errorbar(cut_x[:-1]+[13.4,], cut_y[:-1]+[0,], 
             yerr = cut_y_dev[:-1]+[0,], fmt='.', color = 'k', label = 'Critical T')

plt.xlabel('strain (%)', fontsize=14)
plt.ylabel('Temperature (K)', fontsize=14)
plt.xlim(0,15.5)
plt.ylim(0, 1.9)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(loc = 'upper center')
plt.vlines(13.4, 0, f_Tc(13.4), linestyles ="dashed", colors ="k")
plt.tight_layout()
plt.show()

# save?
# fig3.savefig('/home/sang/UVM/He4HighT/Allfigures/Tc_cut2.png', dpi=400, pad_inches=0.1,)

# To determine the phase boundary, we use 0.6K data

In [ ]:
# Generate dot data for later use
# figure 4: mu-strain at T=0.6K w/ rhos phase diagram
mu_s_rhos = []
 
fig4 = plt.figure('fig4' ,figsize = [6,4])

ph_x = [float(k.split(',')[0])*100 for k in keylst] # strain
ph_y = [float(k.split(',')[1]) for k in keylst] # mu

for k in keylst:
    Ts = featuredata[k][1]
    loc = np.where(np.array(Ts) == 0.6)
    val = float(featuredata[k][6][loc])
    if val < 0:
        val = 0
    mu_s_rhos.append(float(val))

pts = plt.scatter(ph_x, ph_y, c = np.array(mu_s_rhos), cmap = 'magma')
plt.fill_between([0,30], -120 + 1.30*np.array([0,30]), [-120,-120], color = 'lightgray')
plt.xlim(0,30)
plt.ylim(-120,-60)
plt.xlabel('strain (\%)', fontsize=16)
plt.ylabel('chemical potential (K)', fontsize=16)
# plt.text(20, -110, 'Vacuum', fontsize = 24)
# plt.hlines(-101.0, 1, 13, ls = '--', lw = 1.0, label = 'next figure')
cbar = fig3.colorbar(pts)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
fig4.text(0.2, 0.87, 'at 0.6 K', fontsize=14, color='black')
plt.tight_layout()
# plt.legend()
plt.show()

# save?
# fig4.savefig('newfig4.png')

In [ ]:
# figure 5: f-mu strain 12% slice in diagram
vkeys = []
for k in keylst:
    if k[:4] == '0.12':
        vkeys.append(k)
vkeys = vkeys[1:]
fig5 = plt.figure('fig5' ,figsize = [4,3])
markers = ['ko-', 'ko-', 'ks-', 'k*-', 'kD-', 'kp-', 'kP-']
mus = [float(k.split(',')[1]) for k in vkeys]
mus.insert(0, -102.6)
fr = [0,]
frerr = [0,]
sfr = [0,]
sfrerr = [0,]
for k in vkeys:
    Ts = featuredata[k][1]
    loc = np.where(np.array(Ts) == 0.6)
    fr_i = float(featuredata[k][2][loc])
    frerr_i = float(featuredata[k][3][loc])
    sfr_i = float(featuredata[k][4][loc])
    sfrerr_i = float(featuredata[k][5][loc])
    fr.append(fr_i)
    frerr.append(frerr_i)
    sfr.append(sfr_i)
    sfrerr.append(sfrerr_i)

plt.xlim(-103, -88.2)
plt.ylim(0, 0.63)

f_fc = UnivariateSpline(np.array(mus), np.array(fr), k=2)
f_fc.set_smoothing_factor(0.002)
f_sfc = UnivariateSpline(np.array(mus), np.array(sfr), k=2)
f_sfc.set_smoothing_factor(0.002)
new_mus = np.linspace(mus[0],mus[-1],30)
plt.plot(new_mus, f_fc(new_mus), 'k-', linewidth=1.2)
plt.plot(new_mus, f_sfc(new_mus), 'k-', linewidth=1.2)

# plt.scatter(np.array(mus), np.array(fr), s=50, marker='o', color = 'k', facecolors='none', label = 'Density')
plt.errorbar(np.array(mus), np.array(fr), yerr = np.array(frerr),
            fmt='o', elinewidth=1, label='Density', color='k')
plt.errorbar(np.array(mus), np.array(sfr), yerr = np.array(sfrerr),
            fmt='o', elinewidth=1, label='Superfluid', color='coral')

plt.xlabel('chemical potential (K)', fontsize=14)
plt.ylabel('fraction', fontsize=14)
plt.legend()
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
# plt.vlines(-91.8, 0, 0.2)
plt.tight_layout()
fig5.text(0.3, 0.87, 'at 12% strain', fontsize=12, color='black')

plt.show()

# save?
# fig5.savefig('/home/sang/UVM/He4HighT/Allfigures/12_vcut2.png',  dpi=400, pad_inches=0.1,)

# Below codes are hard coded

## type guess and find vanishing point

In [ ]:
# Find boundary SF vanish
guess = -85.7

vkeys = []
for k in keylst:
    if k[:4] == '0.16':
        vkeys.append(k)

fig5 = plt.figure('fig5' ,figsize = [6,4])
markers = ['ko-', 'ko-', 'ks-', 'k*-', 'kD-', 'kp-', 'kP-']
mus = [float(k.split(',')[1]) for k in vkeys]
mus.insert(0, -100)
fr = [0,]
sfr = [0,]
sfrerr = [0,]
for k in vkeys:
    Ts = featuredata[k][1]
    loc = np.where(np.array(Ts) == 0.6)
    fr_i = float(featuredata[k][2][loc])
    sfr_i = float(featuredata[k][4][loc])
    sfrerr_i = float(featuredata[k][5][loc])
    fr.append(fr_i)
    sfr.append(sfr_i)
    sfrerr.append(sfrerr_i)

plt.xlim(-100, -80)
plt.ylim(0, 0.6)
plt.plot(np.array(mus), np.array(fr), 'ko-',
        mfc='none', linewidth=1.2, markersize=8, label='density')
plt.errorbar(np.array(mus), np.array(sfr), yerr = np.array(sfrerr),
            fmt='-', elinewidth=1, capsize=4, label='superfluid')

plt.xlabel('chemical potential (K)', fontsize=16)
plt.ylabel('fraction', fontsize=16)
plt.legend()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.vlines(guess, 0, 0.2)
plt.tight_layout()
fig5.text(0.13, 0.87, 'at 12\% strain', fontsize=14, color='black')

plt.show()

# save?
# fig5.savefig('newfig5.png')

# Following field is for resubmission codes.
## It read 'log' file for continue simulation and make slurm cmd file

In [ ]:
# Make target dataframe for submission
# Example:
target = dbase[(dbase['mu'] == -101.0)&(dbase['estsize'] < 4000)]
print('target df length is: ',len(target))

In [ ]:
# Current date
date = 'May09'
# Filename, date, suffix
gensub.continue_from(target, 'pimc_cont', date, suffix = 0)

## If you copy, transfer, remove files that you want, use this field

In [ ]:
### pick up files what you need
ct = 0
# subset is dataframe you want
subset = target
for idx, row in subset.iterrows():
#     print(row['filedic'])
    !cp /home/sang/UVM/He4Graphene/OUTPUT/gce-*{row['id']}* /home/sang/UVM/He4Graphene/lessthan4k/OUTPUT/
#     !rm ./tosend/gce-*{row['id']}*
    ct += 1
print('Total number of process:', ct)